In [2]:
import os
os.chdir("../../")

In [3]:
import sqlite3
import pandas as pd
from api.utils.database import rows_to_dicts

In [4]:
con = sqlite3.connect("./pipeline/database.db")

### RAW CTA Stops Data

In [4]:
cur = con.cursor()
rows = cur.execute("""
SELECT *
FROM cta_train_stops
LIMIT 5

""").fetchall()

rows

pd.DataFrame(rows_to_dicts(cur, rows))

,stop_id,direction_id,stop_name,station_name,station_descriptive_name,station_id,ada,red,blue,green,brown,purple,purple_express,yellow,pink,orange,longitude,latitude,area_number
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,1,0,0,0,0,0,0,0,1,0,-87.669147,41.857908,31.0
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,1,0,0,0,0,0,0,0,1,0,-87.669147,41.857908,31.0
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,1,0,0,0,0,0,0,0,0,1,-87.680622,41.829353,59.0
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,1,0,0,0,0,0,0,0,0,1,-87.680622,41.829353,59.0
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,1,0,0,1,0,0,0,0,0,0,-87.625826,41.831677,35.0


### RAW CTA Ridership Data

In [5]:
cur = con.cursor()
rows = cur.execute("""
SELECT *
FROM cta_train_ridership


""").fetchall()

rows

pd.DataFrame(rows_to_dicts(cur, rows))

,station_id,stationname,date,daytype,rides
0,40010,Austin-Forest Park,2019-01-01T00:00:00.000,U,576
1,40020,Harlem-Lake,2019-01-01T00:00:00.000,U,981
2,40030,Pulaski-Lake,2019-01-01T00:00:00.000,U,835
3,40040,Quincy/Wells,2019-01-01T00:00:00.000,U,929
4,40050,Davis,2019-01-01T00:00:00.000,U,1207
...,...,...,...,...,...
104559,41660,Lake/State,2020-12-31T00:00:00.000,W,2563
104560,41670,Conservatory,2020-12-31T00:00:00.000,W,425
104561,41680,Oakton-Skokie,2020-12-31T00:00:00.000,W,142
104562,41690,Cermak-McCormick Place,2020-12-31T00:00:00.000,W,312


## ADA vs Non-ADA Daily Stop Ridership Comparison

This looks at average trips a year before COVID and a year after COVID
* Before Covid: March 2019 - Feb 2020
* After Covid: March 2020 - Feb 2021

In [6]:
cur = con.cursor()
rows = cur.execute("""
SELECT 
    c.station_id,
    c.station_name,
    c.ada,
    r.avg_trips_before,
    r.avg_trips_since
FROM cta_train_stops c
    LEFT JOIN (
        SELECT
            station_id,
            stationname,
            SUM(CASE WHEN date < '2021-03-01' THEN rides ELSE 0 END) / 365 AS avg_trips_before,
            SUM(CASE WHEN date >= '2020-03-01' THEN rides ELSE 0 END) / 365 AS avg_trips_since
        FROM cta_train_ridership
        WHERE 
            date >= '2019-03-01'
            AND date < '2021-03-01'
        GROUP BY
            station_id
        ) r
    ON c.station_id = r.station_id
    GROUP BY c.station_id

""").fetchall()

rows

pd.DataFrame(rows_to_dicts(cur, rows))

,station_id,station_name,ada,avg_trips_before,avg_trips_since
0,40010,Austin,0,1831,348
1,40020,Harlem/Lake,1,3913,877
2,40030,Pulaski,1,1703,468
3,40040,Quincy/Wells,0,6711,643
4,40050,Davis,1,3615,609
...,...,...,...,...,...
138,41660,Lake,1,20252,2553
139,41670,Conservatory,1,1117,332
140,41680,Oakton-Skokie,1,918,160
141,41690,Cermak-McCormick Place,1,1876,302


This looks at average trips a year before COVID and a year after COVID at ADA stations vs non ADA stations

In [7]:
cur = con.cursor()
rows = cur.execute("""
SELECT 
    c.ada,
    COUNT(DISTINCT r.station_id) AS num_stations,
    SUM(total_trips_before) / 365 AS avg_trips_before,
    SUM(total_trips_since) / 365 AS avg_trips_since,
    (total_trips_since - total_trips_before) / total_trips_before AS percent_change
FROM cta_train_stops c
    LEFT JOIN (
        SELECT
            station_id,
            CAST(SUM(CASE WHEN date < '2021-03-01' THEN rides ELSE 0 END) AS REAL) AS total_trips_before,
            CAST(SUM(CASE WHEN date >= '2020-03-01' THEN rides ELSE 0 END) AS REAL) AS total_trips_since
        FROM cta_train_ridership
        WHERE 
            date >= '2019-03-01'
            AND date < '2021-03-01'
        GROUP BY
            station_id
        ) r
    ON c.station_id = r.station_id
GROUP BY c.ada

""").fetchall()

rows

pd.DataFrame(rows_to_dicts(cur, rows))

,ada,num_stations,avg_trips_before,avg_trips_since,percent_change
0,0,43,327060.328767,47917.906849,-0.799790
1,1,100,993506.021918,165477.720548,-0.794173


## Questions
**Are there factors, other than an accessible entrance, that a station needs to be ADA compliant? (ie. ramp to get on trian)**

In [8]:
cur = con.cursor()
rows = cur.execute("""
SELECT 
    station_ID,
    COUNT(stop_id) AS num_stops,
    SUM(ada) AS num_ada
FROM cta_train_stops
GROUP BY station_ID
HAVING num_stops != num_ada and num_ada > 0



""").fetchall()

rows

pd.DataFrame(rows_to_dicts(cur, rows))

""


In [9]:
cur = con.cursor()
rows = cur.execute("""
SELECT DISTINCT(station_id)
FROM cta_train_stops




""").fetchall()

rows

pd.DataFrame(rows_to_dicts(cur, rows))

,station_id
0,40830
1,40120
2,41120
3,41270
4,40130
...,...
138,40220
139,41690
140,40540
141,41700


In [10]:
CAST(COUNT(DISTINCT( r.station_id)) AS REAL) / CAST((SUM(DISTINCT(r.station_id))) OVER () AS REAL) AS percentage_of_stations,

SyntaxError: invalid syntax (<ipython-input-10-e26ac8767879>, line 1)

### By community area

In [5]:
cur = con.cursor()
rows = cur.execute("""
SELECT
    c.area_number,
    c.name,
    c.part,
    c.population,
    d.value as disability_rate,
    r.*
FROM community_area c
    LEFT JOIN (
        SELECT 
            area_number,
            SUM(CASE WHEN date < '2021-03-01' THEN rides ELSE 0 END) / 365 AS avg_trips_before,
            SUM(CASE WHEN date >= '2020-03-01' THEN rides ELSE 0 END) / 365 AS avg_trips_since
        FROM (
            SELECT 
                cta_train_ridership.*,
                cta_train_stops.area_number
            FROM cta_train_ridership
            LEFT JOIN cta_train_stops 
                ON cta_train_ridership.station_id = cta_train_stops.station_id          
        )
        WHERE 
            date >= '2019-03-01'
            AND date < '2021-03-01'
        GROUP BY 
            area_number 
    ) r
        ON c.area_number = r.area_number
    LEFT JOIN (
        SELECT *
        FROM disabilities
        WHERE period_end_year = 2019
        AND segment = 'all'
    ) d 
        ON c.area_number = d.area_number

    
    
""").fetchall()

rows

pd.DataFrame(rows_to_dicts(cur, rows))

,area_number,name,part,population,disability_rate,avg_trips_before,avg_trips_since
0,1.0,Rogers Park,Far North Side,52553,10.398003,47512.0,9938.0
1,NaN,West Ridge,Far North Side,78444,9.599939,NaN,NaN
2,3.0,Uptown,Far North Side,53663,11.831111,41210.0,8061.0
3,4.0,Lincoln Square,Far North Side,42033,6.871001,22013.0,3141.0
4,5.0,North Center,North Side,35603,5.511223,10471.0,1254.0
...,...,...,...,...,...,...,...
72,NaN,Washington Heights,Far Southwest Side,26635,17.531878,NaN,NaN
73,NaN,Mount Greenwood,Far Southwest Side,19196,8.845941,NaN,NaN
74,NaN,Morgan Park,Far Southwest Side,21594,15.250978,NaN,NaN
75,76.0,O'Hare,Far North Side,15655,10.950642,33447.0,5801.0


### Burnside

In [ ]:
cur = con.cursor()
rows = cur.execute("""
SELECT * 
FROM cta_train_ridership s
    LEFT JOIN(
        SELECT
            cta_train_stops.*
        FROM cta_train_stops
        LEFT JOIN community_area
                ON cta_train_stops.area_number = community_area.area_number     
    )


""").fetchall()

rows

pd.DataFrame(rows_to_dicts(cur, rows))

In [9]:
cur = con.cursor()
rows = cur.execute("""
SELECT
    c.area_number,
    c.name,
    d.value as disability_rate,
    r.*
FROM community_area c
    LEFT JOIN (
        SELECT 
            area_number,
            SUM(CASE WHEN date < '2021-03-01' THEN rides ELSE 0 END) / 365 AS avg_trips_before,
            SUM(CASE WHEN date >= '2020-03-01' THEN rides ELSE 0 END) / 365 AS avg_trips_since
        FROM (
            SELECT 
                cta_train_ridership.*,
                cta_train_stops.area_number
            FROM cta_train_ridership
            LEFT JOIN cta_train_stops 
                ON cta_train_ridership.station_id = cta_train_stops.station_id          
        )
        WHERE 
            date >= '2019-03-01'
            AND date < '2021-03-01'
        GROUP BY 
            area_number 
    ) r
        ON c.area_number = r.area_number
    LEFT JOIN (
        SELECT *
        FROM disabilities
        WHERE period_end_year = 2019
        AND segment = 'all'
    ) d 
        ON c.area_number = d.area_number
WHERE c.name = "Burnside"
    
    
""").fetchall()

rows

pd.DataFrame(rows_to_dicts(cur, rows))

,area_number,name,disability_rate,avg_trips_before,avg_trips_since
0,None,Burnside,27.848707,None,None
